# JETANK 컨트롤

이 문서는 JEANK 제어 기능을 구축하고 페이지의 버튼을 사용하여 JEANK를 제어하는 데 사용됩니다. 버튼에 대한 기능을 사용자 정의할 수도 있습니다.


### 카메라 클래스 가져오기

다음 코드를 실행하여 카메라 위젯을 엽니다. 화면이 열리면 화면을 마우스 오른쪽 단추로 클릭하고 출력할 새 뷰 만들기를 선택한 다음 화면을 옆의 적절한 위치로 끌어다 놓습니다.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

다음으로 카메라의 Pan 및 Titl 모션을 제어할 수 있는 몇 가지 버튼을 만들고 관련 제어 기능을 설정합니다.
먼저 서보를 제어하는 데 사용되는 라이브러리를 가져오고 2개의 전역 변수를 사용하여 카메라의 수평(Pan) 및 수직(Tilt) 위치를 저장합니다.

In [2]:
from SCSCtrl import TTLServo

servoPos_1 = 0  # Pan position of the camera.
servoPos_5 = 0  # The vertical position of the camera.


def limitCtl(maxInput, minInput, rawInput):
    if rawInput > maxInput:
        limitBuffer = maxInput
    elif rawInput < minInput:
        limitBuffer = minInput
    else:
        limitBuffer = rawInput
        
    return limitBuffer


def cameraUp():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5-15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

def cameraDown():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5+15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

# Camera turn right
def ptRight():
    global servoPos_1
    servoPos_1 = limitCtl(80, -80, servoPos_1+15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)

# Camera turn left
def ptLeft():
    global servoPos_1
    servoPos_1 = limitCtl(80, -80, servoPos_1-15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)

Succeeded to open the port
Succeeded to change the baudrate


버튼 배치를 위한 레이아웃을 추가합니다.

In [3]:
PT_layout = widgets.Layout(width='100px', height='80px', align_self='center')
cameraUp_button = widgets.Button(description='LookUp', button_style='success', layout=PT_layout)
cameraDown_button = widgets.Button(description='LookDown', button_style='success', layout=PT_layout)
ptRight_button = widgets.Button(description='LookRight', button_style='success', layout=PT_layout)
ptLeft_button = widgets.Button(description='LookLeft', button_style='success', layout=PT_layout)

subPT_box = widgets.HBox([ptLeft_button, cameraDown_button, ptRight_button], layout=widgets.Layout(align_self='center'))
surPT_box = widgets.VBox([cameraUp_button, subPT_box])
display(surPT_box)

In [4]:
# Bind the button with the function.
cameraUp_button.on_click(lambda x: cameraUp())
cameraDown_button.on_click(lambda x: cameraDown())
ptRight_button.on_click(lambda x: ptRight())
ptLeft_button.on_click(lambda x: ptLeft())

# 로봇 제어
로봇을 제어하기 위한 몇 개의 버튼을 만드십시오. 다음 코드는 basic_motion에서 가져온 것입니다.

In [5]:
from jetbot import Robot
import time

robot = Robot()

# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
#     time.sleep(0.5)
#     robot.stop()

def step_backward(change):
    robot.backward(0.4)
#     time.sleep(0.5)
#     robot.stop()

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()
    
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

서보를 제어하기 위한 버튼은 다음 코드와 같습니다. 로봇 팔 끝과 그리퍼의 위치를 저장하기 위해서는 xPos, yPos, servoPos_4 세 가지 글로벌 변수가 필요합니다.

In [6]:
xPos = 100
yPos = 0
servoPos_4 = 0

def xIn():
    global xPos
    xPos += 15
    TTLServo.xyInput(xPos, yPos)

def xDe():
    global xPos
    xPos -= 15
    if xPos < 85:
        xPos = 85
    TTLServo.xyInput(xPos, yPos)

def yIn():
    global yPos
    yPos += 15
    TTLServo.xyInput(xPos, yPos)

def yDe():
    global yPos
    yPos -= 15
    TTLServo.xyInput(xPos, yPos)

def grab():
    global servoPos_4
    servoPos_4 -= 15
    if servoPos_4 < -90:
        servoPos_4 = -90
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)
    
def loose():
    global servoPos_4
    servoPos_4+= 15
    if servoPos_4 > -10:
        servoPos_4 = -10
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)

arm_layout = widgets.Layout(width='100px', height='80px', align_self='center')

xIn_button = widgets.Button(description='X+', button_style='success', layout=arm_layout)
xDe_button = widgets.Button(description='X-', button_style='success', layout=arm_layout)

yIn_button = widgets.Button(description='Y+', button_style='success', layout=arm_layout)
yDe_button = widgets.Button(description='Y-', button_style='success', layout=arm_layout)

grab_button = widgets.Button(description='Grab', layout=arm_layout)
loose_button = widgets.Button(description='Loose', layout=arm_layout)

line1_box = widgets.HBox([xIn_button, yIn_button])
line2_box = widgets.HBox([xDe_button, yDe_button])
line3_box = widgets.HBox([grab_button, loose_button])
arm_box = widgets.VBox([line1_box, line2_box, line3_box])

display(arm_box)

버튼을 기능과 함께 바인딩합니다.

In [7]:
xIn_button.on_click(lambda x: xIn())
xDe_button.on_click(lambda x: xDe())

yIn_button.on_click(lambda x: yIn())
yDe_button.on_click(lambda x: yDe())

grab_button.on_click(lambda x: grab())
loose_button.on_click(lambda x: loose())

다시 말씀드리지만, 나중에 노트북에 있는 카메라를 사용할 수 있도록 카메라 커넥터를 제대로 닫도록 하겠습니다.

In [8]:
camera.stop()